### current version

In [1]:
from analyze import *
from modules.llm_eval.consistency_eval import *

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
checklist = Checklist("../../checklist/checklist_sys.csv/", checklist_format=ChecklistFormat.CSV)
extractor = PythonTestFileExtractor(Repository("../../data/raw/openja/lightfm_demo"))

evaluator = TestEvaluator(llm, extractor, checklist)

In [3]:
eval_evaluator = ConsistencyEvaluator()

eval_evaluator.evaluate(models=[{'name': 'model_no1', 'model': evaluator}], num_test_runs=10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:30<00:00, 10.27s/it]


In [4]:
eval_evaluator.get_completeness_score_dist()

test_no,1,2,3,4,5,6,7,8,9,10
model_name,,,,,,,,,,
model_no1,0.3125,0.25,0.3125,0.1875,0.25,0.375,0.25,0.1875,0.25,0.3125


In [5]:
eval_evaluator.get_completeness_score_dist().iloc[0].var()

0.0035156249999999997

In [6]:
eval_evaluator.get_consistency_dist()

test_no           1    2    3    4    5    6    7    8    9   10  consistency
model_name ID                                                                
model_no1  2.1  0.5  0.0  0.0  0.0  0.0  0.5  0.0  0.0  0.5  0.5        False
           3.1  0.0  0.5  0.0  0.0  0.5  0.0  0.0  0.0  0.0  0.0        False
           3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         True
           4.1  1.0  0.5  1.0  0.5  0.5  1.0  1.0  0.5  0.5  1.0        False
           5.1  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5         True
           6.1  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5         True
           6.2  0.0  0.0  0.5  0.0  0.0  0.5  0.0  0.0  0.0  0.0        False
           8.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         True

In [7]:
# import pandas as pd
# pd.DataFrame([r for r in response if r['file'] == '../../data/raw/openja/lightfm_demo/tests/test_data.py'][0]['report']).iloc[4]

### last week's version 

In [8]:
from analyze_prev import TestEvaluator as TestEvaluatorPrev
from modules.llm_eval.consistency_eval import *

In [9]:
evaluator_prev = TestEvaluatorPrev('../../data/raw/openja/lightfm_demo')
evaluator_prev.load_checklist('../../checklist/checklist_sys.csv')

In [10]:
eval_prev_evaluator = ConsistencyEvaluator()

eval_prev_evaluator.evaluate(
    models=[{'name': 'model_no1', 'model': evaluator_prev}], 
    num_test_runs=10, 
    version_before_refactoring=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:30<00:00, 10.21s/it]


In [11]:
eval_prev_evaluator.get_completeness_score_dist()

test_no,1,2,3,4,5,6,7,8,9,10
model_name,,,,,,,,,,
model_no1,0.6875,0.625,0.5,0.625,0.625,0.625,0.625,0.5,0.6875,0.6875


In [12]:
eval_prev_evaluator.get_completeness_score_dist().iloc[0].var()

0.0047309027777777775

In [13]:
eval_prev_evaluator.get_consistency_dist()

test_no           1    2    3    4    5    6    7    8    9   10  consistency
model_name ID                                                                
model_no1  2.1  1.0  1.0  0.5  1.0  1.0  1.0  1.0  0.5  1.0  1.0        False
           3.1  1.0  0.5  0.5  0.5  0.5  0.5  0.5  0.5  1.0  1.0        False
           3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         True
           4.1  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0         True
           5.1  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5         True
           6.1  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0         True
           6.2  1.0  1.0  0.5  1.0  1.0  1.0  1.0  0.5  1.0  1.0        False
           8.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0         True

### F-test to examine the variance difference

In [14]:
var_curr = eval_evaluator.get_completeness_score_dist().iloc[0].var()
var_prev = eval_prev_evaluator.get_completeness_score_dist().iloc[0].var()

In [25]:
df_curr = eval_evaluator.get_completeness_score_dist().shape[1] - 1
df_prev = eval_prev_evaluator.get_completeness_score_dist().shape[1] - 1

In [31]:
F = var_prev / var_curr #var_curr / var_prev if var_prev < var_curr else var_prev / var_curr
F

1.345679012345679

In [17]:
#!pip install scipy

  Using cached scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (60 kB)
Using cached scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl (30.4 MB)


In [32]:
import scipy
alpha = 0.05 #Or whatever you want your alpha to be.
p_value = 1 - scipy.stats.f.cdf(F, df_prev, df_curr) 

print(f"p-value: {p_value}")

if p_value < alpha:
    print("Reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")
else:
    print("Failed to reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")

p-value: 0.33272409885605736
Failed to reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))


### reference
https://www.itl.nist.gov/div898/handbook/eda/section3/eda359.html  
https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/f-test/#:~:text=F%20Test%20to%20Compare%20Two%20Variances,-A%20Statistical%20F&text=If%20the%20variances%20are%20equal,when%20running%20an%20F%20Test  
https://stackoverflow.com/questions/21494141/how-do-i-do-a-f-test-in-python